In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras import backend as K
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
train = pd.read_csv("Train/Train_Data.csv")
test = pd.read_csv("Test/Test_Data.csv")

In [3]:
print(train.shape)
print(test.shape)

(1100, 81)
(360, 80)


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 81 columns):
Id               1100 non-null int64
MSSubClass       1100 non-null int64
MSZoning         1100 non-null object
LotFrontage      908 non-null float64
LotArea          1100 non-null int64
Street           1100 non-null object
Alley            69 non-null object
LotShape         1100 non-null object
LandContour      1100 non-null object
Utilities        1100 non-null object
LotConfig        1100 non-null object
LandSlope        1100 non-null object
Neighborhood     1100 non-null object
Condition1       1100 non-null object
Condition2       1100 non-null object
BldgType         1100 non-null object
HouseStyle       1100 non-null object
OverallQual      1100 non-null int64
OverallCond      1100 non-null int64
YearBuilt        1100 non-null int64
YearRemodAdd     1100 non-null int64
RoofStyle        1100 non-null object
RoofMatl         1100 non-null object
Exterior1st      1100 non-nu

In [ ]:
train.head(n=100)

In [ ]:
train.info()

In [ ]:
output_cols = ["SalePrice"]

XT = train.drop(output_cols,axis=1)
YT = train[output_cols]

print(XT.shape,YT.shape)

In [ ]:
Xt = test

print(Xt.shape)

In [ ]:
from sklearn.base import TransformerMixin

In [ ]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [ ]:
XT = DataFrameImputer().fit_transform(XT)

In [ ]:
print(type(XT))

In [ ]:
XT.head()

In [ ]:
XT.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

one_hot.fit_transform(XT)

In [ ]:
print(type(XT))

In [ ]:
XT.head()

In [ ]:
XT = np.ravel(XT)

In [ ]:
print(XT)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit_transform(XT)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
# XT["Street"] = le.fit_transform(XT["Street"])
# Xt["Street"] = le.fit_transform(Xt["Street"])


XT[:, :] = le.fit_transform(XT[:, :])

In [ ]:
print(XT)

In [ ]:
XT = np.array(XT)
YT = np.array(YT)

Xt = np.array(Xt)

In [ ]:
mean = XT.mean(axis=0)
std = XT.std(axis=0)

# Data Normalisation Step
XT = (XT-mean)/std
Xt = (Xt-mean)/std

In [ ]:
# Build Our Neural Network Model
def r2_score(y_true,y_pred):
    num = K.sum(K.square(y_true-y_pred))
    denom = K.sum(K.square(y_true- K.mean(y_true)))
    return (1-num/denom)

def getModel():
    model = Sequential()
    model.add(Dense(64,activation='relu',input_shape=(80,)))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam',loss='mse',metrics=[r2_score])
    return model

In [ ]:
model = getModel()
model.summary()

In [ ]:
hist = model.fit(XT,YT,verbose=1,epochs=100)

In [ ]:
# val_loss = hist.history['val_loss']
train_loss = hist.history['loss']

In [ ]:
import matplotlib.pyplot as plt

# plt.plot(val_loss,label="Val Loss ")
plt.plot(train_loss,label="Train Loss")
plt.legend()
plt.show()